Import Libraries and Setup

In [ ]:
# Import necessary libraries
import pandas as pd
import plotly.graph_objects as go
from prophet import Prophet
from prophet.plot import plot_plotly, plot_components_plotly
from datetime import datetime, timedelta
from sklearn.metrics import mean_absolute_error
import pickle
import warnings
import os

# Suppress warnings and format floats for readability
warnings.filterwarnings('ignore')
pd.options.display.float_format = '${:,.2f}'.format

# Define a directory to save models
MODEL_DIR = 'models'
os.makedirs(MODEL_DIR, exist_ok=True)

Load and Prepare Data

In [ ]:
# Load the dataset from the CSV file
df = pd.read_csv('btc_usd_history.csv')

# Select and rename columns for Prophet
df1 = df[['Date', 'Open']].rename(columns={'Date': 'ds', 'Open': 'y'})

# Ensure 'ds' is in datetime format and timezone-naive
df1['ds'] = pd.to_datetime(df1['ds'])
if df1['ds'].dt.tz is not None:
    df1['ds'] = df1['ds'].dt.tz_localize(None)

# Sort the data by date
df1 = df1.sort_values('ds')

# Filter data to focus on recent years (e.g., last 5 years) for better short-term accuracy
df1 = df1[df1['ds'] >= '2020-01-01']  # Adjust based on your data availability

Visualize Bitcoin Price Time Series

In [3]:
# Visualize the Bitcoin price time series
fig = go.Figure()
fig.add_trace(go.Scatter(x=df1['ds'], y=df1['y'], mode='lines', name='Bitcoin Open Price'))
fig.update_layout(
    title='Bitcoin Open Price Time Series (Recent Data)',
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1, label='1m', step='month', stepmode='backward'),
                dict(count=6, label='6m', step='month', stepmode='backward'),
                dict(count=1, label='1y', step='year', stepmode='backward'),
                dict(step='all')
            ])
        ),
        rangeslider=dict(visible=True),
        type='date'
    )
)
fig.show()

Load or Train the Model

In [4]:
# Define the model file path
model_file = os.path.join(MODEL_DIR, 'prophet_bitcoin_model.pkl')

# Check if the model file exists to load it, otherwise train and save
if os.path.exists(model_file):
    print(f"Loading existing model from {model_file}")
    with open(model_file, 'rb') as f:
        model = pickle.load(f)
else:
    print("Training new model...")
    # Initialize Prophet with tuned parameters for volatile data
    model = Prophet(
        seasonality_mode='additive',  # Switch to additive for volatile data
        changepoint_prior_scale=0.5,  # Increase flexibility for trend changes
        seasonality_prior_scale=1.0,  # Reduce seasonality strength
        yearly_seasonality=True,
        weekly_seasonality=True,
        daily_seasonality=False
    )
    model.fit(df1)
    
    # Save the trained model
    with open(model_file, 'wb') as f:
        pickle.dump(model, f)
    print(f"Model saved to {model_file}")

Loading existing model from models\prophet_bitcoin_model.pkl


Make Predictions

In [5]:
# Define the start date for prediction (today is April 6, 2025, predict for April 7, 2025)
start_date = datetime(2025, 4, 7)  # Adjusted to predict the next day from today

# Create future dates for the next 7 days (short-term focus)
future = pd.DataFrame({'ds': pd.date_range(start=start_date, periods=7, freq='D')})
forecast = model.predict(future)

# Display the predictions
print("Forecasted Values (April 7, 2025 - April 13, 2025):")
print(forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']])

# Predict the next day's price (April 7, 2025)
next_day = start_date.strftime('%Y-%m-%d')
predicted_value = forecast.loc[forecast['ds'] == next_day, 'yhat'].iloc[0]
print(f"\nPredicted Bitcoin Open Price for {next_day}: ${predicted_value:,.2f}")

Forecasted Values (April 7, 2025 - April 13, 2025):
          ds        yhat  yhat_lower  yhat_upper
0 2025-04-07 $110,986.88 $105,233.95 $116,240.77
1 2025-04-08 $111,713.82 $106,505.49 $117,096.14
2 2025-04-09 $112,367.70 $107,040.21 $118,091.82
3 2025-04-10 $113,255.41 $108,048.75 $118,582.64
4 2025-04-11 $113,289.30 $107,888.21 $118,517.52
5 2025-04-12 $113,709.44 $108,604.44 $119,140.48
6 2025-04-13 $113,840.33 $108,501.35 $118,786.56

Predicted Bitcoin Open Price for 2025-04-07: $110,986.88


Visualize the Forecast

In [ ]:
# Visualize the short-term forecast
fig_forecast = plot_plotly(model, forecast)
fig_forecast.update_layout(title='Bitcoin Open Price Forecast (April 7, 2025 - April 13, 2025)')
fig_forecast.show()

Additional Graph - Forecast Components

In [7]:
# Plot the components of the forecast
fig_components = plot_components_plotly(model, forecast)
fig_components.show()

Model Evaluation

In [ ]:
# Evaluate the model using a train-test split on recent data
train = df1[df1['ds'] < '2024-01-01']
test = df1[df1['ds'] >= '2024-01-01']

# Define the evaluation model file path
eval_model_file = os.path.join(MODEL_DIR, 'prophet_bitcoin_eval_model.pkl')

# Check if evaluation model file exists, otherwise train and save
if os.path.exists(eval_model_file):
    print(f"Loading existing evaluation model from {eval_model_file}")
    with open(eval_model_file, 'rb') as f:
        eval_model = pickle.load(f)
else:
    print("Training new evaluation model...")
    eval_model = Prophet(
        seasonality_mode='additive',
        changepoint_prior_scale=0.5,
        seasonality_prior_scale=1.0,
        yearly_seasonality=True,
        weekly_seasonality=True,
        daily_seasonality=False
    )
    eval_model.fit(train)
    
    # Save the trained evaluation model
    with open(eval_model_file, 'wb') as f:
        pickle.dump(eval_model, f)
    print(f"Evaluation model saved to {eval_model_file}")

# Make predictions on the test period
future_test = eval_model.make_future_dataframe(periods=len(test))
forecast_test = eval_model.predict(future_test)

# Calculate Mean Absolute Error (MAE)
mae = mean_absolute_error(test['y'], forecast_test['yhat'][-len(test):])
print(f"Mean Absolute Error on Test Set: ${mae:,.2f}")

# Suggest parameter tuning if MAE is too high
if mae > 10000:  # Arbitrary threshold, adjust as needed
    print("MAE is high. Consider further tuning changepoint_prior_scale or adding external regressors.")

Loading existing evaluation model from models\prophet_bitcoin_eval_model.pkl
Mean Absolute Error on Test Set: $73,920.39
MAE is high. Consider further tuning changepoint_prior_scale or adding external regressors.


Additional Graph - Actual vs. Predicted

In [9]:
# Plot actual vs predicted for the test period
fig_test = go.Figure()
fig_test.add_trace(go.Scatter(x=test['ds'], y=test['y'], mode='lines', name='Actual'))
fig_test.add_trace(go.Scatter(x=test['ds'], y=forecast_test['yhat'][-len(test):], mode='lines', name='Predicted'))
fig_test.update_layout(title='Actual vs Predicted Bitcoin Open Price (Test Set)')
fig_test.show()